# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 15 2023 11:04AM,257448,19,ADV80011892,"AdvaGen Pharma, Ltd",Released
1,Feb 15 2023 11:04AM,257448,19,ADV80011893,"AdvaGen Pharma, Ltd",Released
2,Feb 15 2023 11:04AM,257448,19,ADV80011894,"AdvaGen Pharma, Ltd",Released
3,Feb 15 2023 11:04AM,257448,19,ADV80011895,"AdvaGen Pharma, Ltd",Released
4,Feb 15 2023 11:04AM,257448,19,ADV80011959,"AdvaGen Pharma, Ltd",Released
5,Feb 15 2023 11:04AM,257448,19,ADV80011960,"AdvaGen Pharma, Ltd",Released
6,Feb 15 2023 11:04AM,257448,19,ADV80011961,"AdvaGen Pharma, Ltd",Released
7,Feb 15 2023 11:04AM,257448,19,ADV80011962,"AdvaGen Pharma, Ltd",Released
8,Feb 15 2023 11:04AM,257448,19,ADV80011963,"AdvaGen Pharma, Ltd",Released
9,Feb 15 2023 11:04AM,257448,19,ADV80011950,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
47,257446,Executing,5
48,257446,Released,6
49,257447,Released,3
50,257448,Released,45
51,257449,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257444,NaN,NaN,32.0
257446,NaN,5.0,6.0
257447,NaN,NaN,3.0
257448,NaN,NaN,45.0
257449,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257349,0.0,0.0,3.0
257352,0.0,0.0,1.0
257353,0.0,0.0,1.0
257354,1.0,0.0,6.0
257366,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257349,0,0,3
257352,0,0,1
257353,0,0,1
257354,1,0,6
257366,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257349,0,0,3
1,257352,0,0,1
2,257353,0,0,1
3,257354,1,0,6
4,257366,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257349,,,3
1,257352,,,1
2,257353,,,1
3,257354,1,,6
4,257366,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 15 2023 11:04AM,257448,19,"AdvaGen Pharma, Ltd"
45,Feb 15 2023 10:58AM,257449,10,"CLINUVEL, Inc."
46,Feb 15 2023 10:49AM,257447,10,Eywa Pharma Inc.
49,Feb 15 2023 10:48AM,257424,10,Eywa Pharma Inc.
55,Feb 15 2023 10:40AM,257430,19,Eywa Pharma Inc.
56,Feb 15 2023 10:36AM,257446,10,Bio-PRF
67,Feb 15 2023 10:35AM,257442,10,ISDIN Corporation
105,Feb 15 2023 10:34AM,257444,10,ISDIN Corporation
137,Feb 15 2023 10:34AM,257441,10,ISDIN Corporation
166,Feb 15 2023 10:34AM,257440,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 15 2023 11:04AM,257448,19,"AdvaGen Pharma, Ltd",,,45
1,Feb 15 2023 10:58AM,257449,10,"CLINUVEL, Inc.",,,1
2,Feb 15 2023 10:49AM,257447,10,Eywa Pharma Inc.,,,3
3,Feb 15 2023 10:48AM,257424,10,Eywa Pharma Inc.,,,6
4,Feb 15 2023 10:40AM,257430,19,Eywa Pharma Inc.,,,1
5,Feb 15 2023 10:36AM,257446,10,Bio-PRF,,5,6
6,Feb 15 2023 10:35AM,257442,10,ISDIN Corporation,,,38
7,Feb 15 2023 10:34AM,257444,10,ISDIN Corporation,,,32
8,Feb 15 2023 10:34AM,257441,10,ISDIN Corporation,,,29
9,Feb 15 2023 10:34AM,257440,10,ISDIN Corporation,,,28


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 11:04AM,257448,19,"AdvaGen Pharma, Ltd",45,,
1,Feb 15 2023 10:58AM,257449,10,"CLINUVEL, Inc.",1,,
2,Feb 15 2023 10:49AM,257447,10,Eywa Pharma Inc.,3,,
3,Feb 15 2023 10:48AM,257424,10,Eywa Pharma Inc.,6,,
4,Feb 15 2023 10:40AM,257430,19,Eywa Pharma Inc.,1,,
5,Feb 15 2023 10:36AM,257446,10,Bio-PRF,6,5,
6,Feb 15 2023 10:35AM,257442,10,ISDIN Corporation,38,,
7,Feb 15 2023 10:34AM,257444,10,ISDIN Corporation,32,,
8,Feb 15 2023 10:34AM,257441,10,ISDIN Corporation,29,,
9,Feb 15 2023 10:34AM,257440,10,ISDIN Corporation,28,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 11:04AM,257448,19,"AdvaGen Pharma, Ltd",45,,
1,Feb 15 2023 10:58AM,257449,10,"CLINUVEL, Inc.",1,,
2,Feb 15 2023 10:49AM,257447,10,Eywa Pharma Inc.,3,,
3,Feb 15 2023 10:48AM,257424,10,Eywa Pharma Inc.,6,,
4,Feb 15 2023 10:40AM,257430,19,Eywa Pharma Inc.,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 11:04AM,257448,19,"AdvaGen Pharma, Ltd",45.0,NaN,NaN
1,Feb 15 2023 10:58AM,257449,10,"CLINUVEL, Inc.",1.0,NaN,NaN
2,Feb 15 2023 10:49AM,257447,10,Eywa Pharma Inc.,3.0,NaN,NaN
3,Feb 15 2023 10:48AM,257424,10,Eywa Pharma Inc.,6.0,NaN,NaN
4,Feb 15 2023 10:40AM,257430,19,Eywa Pharma Inc.,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 15 2023 11:04AM,257448,19,"AdvaGen Pharma, Ltd",45.0,0.0,0.0
1,Feb 15 2023 10:58AM,257449,10,"CLINUVEL, Inc.",1.0,0.0,0.0
2,Feb 15 2023 10:49AM,257447,10,Eywa Pharma Inc.,3.0,0.0,0.0
3,Feb 15 2023 10:48AM,257424,10,Eywa Pharma Inc.,6.0,0.0,0.0
4,Feb 15 2023 10:40AM,257430,19,Eywa Pharma Inc.,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,7464905,247.0,12.0,0.0
12,514854,1.0,1.0,0.0
15,1287052,36.0,21.0,1.0
19,1544464,49.0,2.0,0.0
21,514865,2.0,0.0,0.0
22,772178,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,7464905,247.0,12.0,0.0
1,12,514854,1.0,1.0,0.0
2,15,1287052,36.0,21.0,1.0
3,19,1544464,49.0,2.0,0.0
4,21,514865,2.0,0.0,0.0
5,22,772178,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,247.0,12.0,0.0
1,12,1.0,1.0,0.0
2,15,36.0,21.0,1.0
3,19,49.0,2.0,0.0
4,21,2.0,0.0,0.0
5,22,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,247.0
1,12,Released,1.0
2,15,Released,36.0
3,19,Released,49.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21,22
Status,,,,,,
Completed,0.0,0.0,1.0,0.0,0.0,0.0
Executing,12.0,1.0,21.0,2.0,0.0,2.0
Released,247.0,1.0,36.0,49.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21,22
0,Completed,0.0,0.0,1.0,0.0,0.0,0.0
1,Executing,12.0,1.0,21.0,2.0,0.0,2.0
2,Released,247.0,1.0,36.0,49.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21,22
0,Completed,0.0,0.0,1.0,0.0,0.0,0.0
1,Executing,12.0,1.0,21.0,2.0,0.0,2.0
2,Released,247.0,1.0,36.0,49.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()